In [ ]:
!pip install bert-extractive-summarizer
!pip install transformers
from google.colab import drive
drive.mount('/content/drive')
nlp_folder_path = '/content/drive/MyDrive/Natural Language Processing/'

In [ ]:
import json
import os
count = 0
dataset = []
for fj in os.listdir(nlp_folder_path+"dataset"):
  with open(nlp_folder_path+ "dataset/" + fj, encoding="utf8") as f:
    data = json.load(f)

    if data["Başlık"]!="" and data["Özet"]!="" and data["Metin"]!="":
      instance = {"id":fj[:-5],"title":data["Başlık"],"abstract":data["Özet"],"text":data["Metin"]}
      dataset.append(instance)

In [ ]:
processed=[ a[15:-5] for a in os.listdir(nlp_folder_path+"summary_results_2")]

In [ ]:
processed

In [ ]:
from posixpath import split
from summarizer import TransformerSummarizer
import time 

GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-large")

for data in dataset:

  if data["id"] not in processed:
    start = time.time()
    
    print(data["id"])        
    body = data["text"][:1000000]
    
    # ''.join(data["text"].split(" ")[:10000])
    
    full = ''.join(GPT2_model(body, min_length=60,max_length=len(data["abstract"])))
    output = {"id":data["id"],"summary":full}

    # Save the results in same folder as the preprocessed dataset
    if not os.path.exists(nlp_folder_path+"summary_results_2"):
        os.makedirs(nlp_folder_path+"summary_results_2")
    with open(nlp_folder_path+"summary_results_2/summary_result_{}.json".format(data["id"]), "w", encoding="utf8") as f:
        json.dump(output, f, ensure_ascii=False)

    print("Summary for document {} are extracted".format(data["id"]))
    print("Time elapsed: {}".format(time.time() - start))

In [ ]:
!pip install rouge
import rouge
import os,json

def compare_rouge(hypothesis, reference):
    scorer = rouge.Rouge()
    scores = scorer.get_scores(hypothesis, reference)
    return scores

In [ ]:
# list dir summary_results_2
filenames = os.listdir(nlp_folder_path+ 'summary_results_2')
article_ids = []
# for each file in dir do substr [15:-5]
for filename in filenames:
    article_ids.append( filename[15:-5])

for article in article_ids:
    summary ={}
    reference = {}

    # read summary_results_2/summary_result_{}.json
    with open(nlp_folder_path+'summary_results_2/summary_result_{}.json'.format(article), 'r') as f:
        summary = json.load(f)

    # read dataset/{}.json
    with open(nlp_folder_path+ 'dataset/{}.json'.format(article), 'r') as f:
        reference = json.load(f)

    # compare_rouge(summary, reference)
    scores = compare_rouge(summary['summary'], reference['Özet'])

    summary['scores'] = scores

    # write summary_results_2/summary_result_{}.json
    with open(nlp_folder_path+'summary_results_2/summary_result_{}.json'.format(article), 'w') as f:
        json.dump(summary, f)